In [1]:
!pip install -q langchain langchain-nvidia-ai-endpoints gradio
!pip install gradio
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.6/314.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 1.6 MB/s et

In [2]:
from getpass import getpass
import requests
import os
from typing import List, Union, Iterable, Dict, Optional
from operator import itemgetter
from functools import partial

from langchain_core.runnables import RunnableLambda

from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_nvidia_ai_endpoints._common import NVEModel

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, ChatMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticOutputParser

from langchain.pydantic_v1 import BaseModel, Field

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain.schema.runnable import (RunnableBranch, RunnableLambda,RunnableMap,       ## Wrap an implicit "dictionary" runnable
                                       RunnablePassthrough,
)
import gradio as gr

from transformers import pipeline


In [3]:


# Assign your NVIDIA API key directly here
os.environ["NVIDIA_API_KEY"] = "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"

# Check if the NVIDIA_API_KEY environment variable has been set properly
if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
    print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")
else:
    print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")

# Now you can proceed with the rest of your code
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models


Retrieved NVIDIA_API_KEY beginning with "nvapi-VAO..."


{'playground_nvolveqa_40k': '091a03bb-7364-4087-8090-bd71e9277520',
 'ai-gemma-2b': '04174188-f742-4069-9e72-d77c2b77d3cb',
 'ai-embed-qa-4': '09c64e32-2b65-4892-a285-2f585408d118',
 'ai-rerank-qa-mistral-4b': '0bf77f50-5c35-4488-8e7a-f49bb1974af6',
 'playground_gemma_7b': '1361fa56-61d7-4a12-af32-69a3825746fa',
 'playground_llama2_70b': '0e349b44-440a-44e1-93e9-abe8dcb27158',
 'ai-arctic-embed-l': '1528a0ad-205a-46ac-a783-94e2372586a9',
 'playground_nemotron_qa_8b': '0c60f14d-46cb-465e-b994-227e1c3d5047',
 'playground_llama2_code_70b': '2ae529dc-f728-4a46-9b8d-2697213666d8',
 'playground_mamba_chat': '381be320-4721-4664-bd75-58f8783b43c7',
 'ai-recurrentgemma-2b': '2f495340-a99f-4b4b-89bd-1beb003dd896',
 'playground_mistral_7b': '35ec3354-2681-4d0e-a8dd-80325dcf7c63',
 'playground_starcoder2_15b': '6acada03-fe2f-4e4d-9e0a-e711b9fd1b59',
 'ai-dbrx-instruct': '3d6c2ff8-8bfc-4d10-8fd0-b7337288e869',
 'ai-microsoft-kosmos-2': '6018fed7-f227-48dc-99bc-3fd4264d5037',
 'ai-llama2-70b': '2fdd

In [4]:
'''hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try:
        assert not hard_reset
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = "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"#response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models'''

'hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY\nwhile "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:\n    try:\n        assert not hard_reset\n        response = requests.get("http://docker_router:8070/get_key").json()\n        assert response.get(\'nvapi_key\')\n    except: response = {\'nvapi_key\' : getpass("NVIDIA API Key: ")}\n    os.environ["NVIDIA_API_KEY"] = "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"#response.get("nvapi_key")\n    try: requests.post("http://docker_router:8070/set_key/", json={\'nvapi_key\' : os.environ["NVIDIA_API_KEY"]}).json()\n    except: pass\n    hard_reset = False\n    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):\n        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")\n\nprint(f"Retrieved NVIDIA_API_KEY beginning with "{os.environ.get(\'NVIDIA_API_KEY\')[:9]}..."")\nfrom langchain_nvidia_ai_endpoint

In [5]:
first_prompt=ChatPromptTemplate.from_messages([
    ("system",("[Option: {options}] and please tell the reason of choice in single sentence")),
    ("user","{input}=")

])

def EnumParser(*idxs):
    '''Method that pulls out values from a mistral model that outputs numbered entries'''
    idxs = idxs or [slice(0, None, 1)]
    entry_parser = lambda v: v if (' ' not in v) else v[v.index(' '):]
    out_lambda = lambda x: [entry_parser(v).strip() for v in x.split("\n")]
    return StrOutputParser() | RunnableLambda(lambda x: itemgetter(*idxs)(out_lambda(x)))

model_llm=ChatNVIDIA(model="mistral_7b") | EnumParser(0)
first_chain=first_prompt | model_llm
def to_run(input,options=["Car","Bike","Bus","Autorikshaw"]):
  return first_chain.invoke({"input":input, "options":options})

print("-" * 80)
print(to_run("which one to go with and why?"))

print("-" * 80)
print(to_run("I get motionsick, so I think I'll pass on the trip"))

print("-" * 80)
print(to_run("I'm scared of high speed, so high speed probably isn't for me"))


--------------------------------------------------------------------------------
on the given context, if you're looking for a personal vehicle option that is likely to have less traffic congestion and offer more flexibility in terms of parking, then a Bike would be a good choice.
--------------------------------------------------------------------------------
my sensitivity to motion sickness, I would prefer not to travel by Car, Bike, Bus, or Autorikshaw for this trip.
--------------------------------------------------------------------------------
my fear of high speed, I would likely choose an Autorikshaw or Bus over a Car or Bike.


In [6]:
second_prompt=ChatPromptTemplate.from_template(
    "Make a new short roamantic sentence about the the following topic: : {topic}. Be creative!"
)

second_chain=second_prompt | model_llm

inp_msg="I get motionsick, so I think I'll pass on the trip"
options=["Car","Bike","Bus","Autorikshaw"]

({'topic':first_chain}|second_chain).invoke({"input":inp_msg, "options":options})

'my heart yearning for adventure, my delicate senses to motion make me long for a tranquil journey by train or plane for our upcoming trip."'

In [7]:
def RPrint(preface=""):
  def print_and_return(x, preface=""):
    print(f"{preface}{x}")
    return x
  return RunnableLambda(partial(print_and_return, preface=preface))

final_chain=(
    RPrint("State:") |
    {'input': lambda d: d.get('input'),'topic': first_chain} |
    RPrint("State:") |
    RunnableAssign({'generation': second_chain}) |
    RPrint("State:") |
    RunnableAssign({ 'combination': (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        ) | model_llm
    )
    })
)

final_chain.invoke({
    "input" : "I get motionsick, so I think I'll pass on the trip",
    "options" : ["Car","Bike","Bus","Autorikshaw"]
})

State:{'input': "I get motionsick, so I think I'll pass on the trip", 'options': ['Car', 'Bike', 'Bus', 'Autorikshaw']}
State:{'input': "I get motionsick, so I think I'll pass on the trip", 'topic': 'my sensitivity to motion sickness, I would prefer not to travel by Car, Bike, Bus, or Autorikshaw for this trip.'}
State:{'input': "I get motionsick, so I think I'll pass on the trip", 'topic': 'my sensitivity to motion sickness, I would prefer not to travel by Car, Bike, Bus, or Autorikshaw for this trip.', 'generation': 'my heart yearning for adventure, my delicate senses to motion make me long for a tranquil journey by train or plane for our upcoming trip."'}


{'input': "I get motionsick, so I think I'll pass on the trip",
 'topic': 'my sensitivity to motion sickness, I would prefer not to travel by Car, Bike, Bus, or Autorikshaw for this trip.',
 'generation': 'my heart yearning for adventure, my delicate senses to motion make me long for a tranquil journey by train or plane for our upcoming trip."',
 'combination': "I get motion sick, I'd prefer a peaceful journey by train or plane for your upcoming trip, as my heart yearns for adventure but my senses are sensitive to motion."}

# Part 2

To build a responsive and intelligent system, we need a method that not only processes inputs but also retains and updates essential information through the flow of conversation.

This is where the combination of LangChain and Pydantic becomes pivotal. [**Pydantic**](https://docs.pydantic.dev/latest/), a popular Python validation library, is instrumental in structuring and validating data models.

As one of its features, Pydantic offers structured "model" classes that validate objects (data, classes, themselves, etc.) with simplified syntax and deep rabbitholes of customization options. This framework is used throughout LangChain and comes up as a necessary component for use cases that involve data coersion.

Think of [**Pydantic**] as a tool that helps you define the structure and rules for your data. It ensures that the data you receive or process follows certain guidelines you set, like a gatekeeper checking incoming information to make sure it's correct and fits the expected format.

In [8]:
#importing BaseModel and Field class to create a structure Knowledge Base
class Knowledgebase(BaseModel):
      ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
      topic: str= Field('general', description="Current conversation topic")
      user_preference: Dict[str, Union[str,int]]= Field({}, description="User preferences and choices")
      session_notes: str= Field("", description="Notes on the ongoing session")
      unresolved_queries: list= Field([], description="Notes on the ongoing session")
      action_items: list= Field([], description="Actionable items identified during the conversation")

print(repr(Knowledgebase(topic = "Travel")))


Knowledgebase(topic='Travel', user_preference={}, session_notes='', unresolved_queries=[], action_items=[])


This functionality generates instructions for creating valid inputs to the Knowledge Base, which in turn helps the LLM by providing a concrete one-shot example of the desired output format.

In [9]:
instruct_string = PydanticOutputParser(pydantic_object=Knowledgebase).get_format_instructions()
print(instruct_string)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"topic": {"title": "Topic", "description": "Current conversation topic", "default": "general", "type": "string"}, "user_preference": {"title": "User Preference", "description": "User preferences and choices", "default": {}, "type": "object", "additionalProperties": {"anyOf": [{"type": "string"}, {"type": "integer"}]}}, "session_notes": {"title": "Session Notes", "description": "Notes on the ongoing session", "default": "", "type": "string"}, "unresolved_queries": {"title": "Unresolved Queries", "description": "Notes on the ongo

Now in this part we are going to preprocess and extract informations from the text. This is a pipiline to do the same while holding the proper format.

In [10]:
def RExtract(pydantic_strcture, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser=PydanticOutputParser(pydantic_object=pydantic_strcture) #handles the extraction and validation of information based on the structure defined by the Pydantic class
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    #for preprocessing
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        return string
    return instruct_merge | prompt | llm | preparse | parser

In [11]:
def RExtract(pydantic_strcture, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser=PydanticOutputParser(pydantic_object=pydantic_strcture) #handles the extraction and validation of information based on the structure defined by the Pydantic class
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    #for preprocessing
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        return string
    return instruct_merge | prompt | llm | preparse | parser

instruct_model=ChatNVIDIA(model='mixtral_8x7b') | StrOutputParser()
parse_prompt= ChatPromptTemplate.from_messages([
    ("system", "Update the knowledge base: {format_instructions}. Only use information from the input."),
    ("user", "{input}")
])
extractor = RExtract(Knowledgebase, instruct_model, parse_prompt)

knowledge = extractor.invoke({'input' : "I love flowers so much! The orchids are amazing! I want to buy some?"})
knowledge

Knowledgebase(topic='flowers', user_preference={}, session_notes='User is interested in buying orchids.', unresolved_queries=[], action_items=[{'task': 'recommend places to buy orchids'}])

Do keep in mind that this process can fail due to the fuzzy nature of LLM prediction, especially with models that are not optimized for instruction-following! For this process, it's important to have a strong instruction-following LLM with extra checks and graceful failure routines.

In [12]:
class Knowledgebase(BaseModel):
    firstname: str = Field('unknown', description='Chatting with  firstname and if the name not known make it unknown')
    lastlename: str = Field('unknown', description='Chatting with lastname and if the name not known make it unknown')
    location: str = Field('unknown', description='Where is the location')
    hints: str = Field('unknown', description="Hints to make the system understand the location")
    response: str = Field('unknown', description= "Ideal response based on last user input")

instruct_model=ChatNVIDIA(model='mixtral_8x7b') | StrOutputParser()

parse_prompt= ChatPromptTemplate.from_messages([
    ("system", (
        "The user just responsed. Please update the knowledge base based on the response."
        " This information will be acted on to respond to the user in the next interaction."
        " Do not hallucinate any details, and make sure the knowledge base is not redundant."
        " Update the entries frequently to adapt to the conversation flow."
        "\n{format_instructions}")),
    ("user", "CURRENT KNOWLEDGE BASE:{know_base}\nUser:{input}"),
])

extractor = RExtract(Knowledgebase, instruct_model, parse_prompt)
info_update = RunnableAssign({'know_base' : extractor})

state={'know_base': Knowledgebase()}
state['input'] = "My name is Carmen Sandiego! Guess where I am?"
state = info_update.invoke(state)
print(state)

state['input'] = "The United States is a big place! Can you be more specific?"
state = info_update.invoke(state)
print(state)

state['input'] = "Yeah, I'm in New Orleans... How did you know?"
state = info_update.invoke(state)
print(state)

{'know_base': Knowledgebase(firstname='Carmen', lastlename='Sandiego', location='unknown', hints='Guess where I am?', response="That's an interesting place to be, Carmen Sandiego! Let's continue our conversation."), 'input': 'My name is Carmen Sandiego! Guess where I am?'}
{'know_base': Knowledgebase(firstname='Carmen', lastlename='Sandiego', location='The United States', hints='Guess where I am?', response='The United States is a big place indeed, Carmen Sandiego! Could you please provide a more specific location? It would help me continue our conversation.'), 'input': 'The United States is a big place! Can you be more specific?'}
{'know_base': Knowledgebase(firstname='Carmen', lastlename='Sandiego', location='New Orleans', hints='Guess where I am?', response='Ah, New Orleans! I had a hunch based on your previous hint. How can I assist you today in the Big Easy, Carmen?'), 'input': "Yeah, I'm in New Orleans... How did you know?"}


This example demonstrates how a running state chain can be effectively utilized to manage a conversation with evolving context and requirements, making it a powerful tool for developing sophisticated interactive systems.

# Flight Chat Bot

In [13]:
def flight_info(d:dict)->str:
  """
  Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
  """
  req_key=['firstname','lastname','confirmation_pin'] #this is the require key
  #it is checking the condition if it is true and if all the key are present in d then it will print it
  #if false then it will ask for the required key by showing the original dictionary d
  assert all((key in d) for key in req_key), f"Expected dictionary with keys {req_key}, got {d}"

  more_key=req_key+["departure", "destination", "departure_time", "arrival_time", "flight_day"]
  values=[
        ["Jane", "Doe", 12345, "San Jose", "New Orleans", "12:30 PM", "9:30 PM", "tomorrow"],
        ["John", "Smith", 54321, "New York", "Los Angeles", "8:00 AM", "11:00 AM", "Sunday"],
        ["Alice", "Johnson", 98765, "Chicago", "Miami", "7:00 PM", "11:00 PM", "next week"],
        ["Bob", "Brown", 56789, "Dallas", "Seattle", "1:00 PM", "4:00 PM", "yesterday"],
    ]
  #to avoid writing a function for concat we are using this to make concat
  get_key = lambda d: "|".join([d['firstname'], d['lastname'], str(d['confirmation_pin'])])
  #creating a key value pair
  get_val = lambda l: {k:v for k,v in zip(more_key, l)}
  db = {get_key(get_val(entry)) : get_val(entry) for entry in values}
  data = db.get(get_key(d))

  if not data:
        return (
            f"Based on {req_key} = {get_key(d)}) from your knowledge base, no info on the user flight was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
  return (
        f"{data['firstname']} {data['lastname']}'s flight from {data['departure']} to {data['destination']}"
        f" departs at {data['departure_time']} {data['flight_day']} and lands at {data['arrival_time']}."
    )

In [14]:
def flight_info(d:dict)->str:
  """
  Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
  """
  req_key=['firstname','lastname','confirmation_pin'] #this is the require key
  #it is checking the condition if it is true and if all the key are present in d then it will print it
  #if false then it will ask for the required key by showing the original dictionary d
  assert all((key in d) for key in req_key), f"Expected dictionary with keys {req_key}, got {d}"

  more_key=req_key+["departure", "destination", "departure_time", "arrival_time", "flight_day"]
  values=[
        ["Jane", "Doe", 12345, "San Jose", "New Orleans", "12:30 PM", "9:30 PM", "tomorrow"],
        ["John", "Smith", 54321, "New York", "Los Angeles", "8:00 AM", "11:00 AM", "Sunday"],
        ["Alice", "Johnson", 98765, "Chicago", "Miami", "7:00 PM", "11:00 PM", "next week"],
        ["Bob", "Brown", 56789, "Dallas", "Seattle", "1:00 PM", "4:00 PM", "yesterday"],
    ]
  #to avoid writing a function for concat we are using this to make concat
  get_key = lambda d: "|".join([d['firstname'], d['lastname'], str(d['confirmation_pin'])])
  #creating a key value pair
  get_val = lambda l: {k:v for k,v in zip(more_key, l)}
  db = {get_key(get_val(entry)) : get_val(entry) for entry in values}
  data = db.get(get_key(d))

  if not data:
        return (
            f"Based on {req_key} = {get_key(d)}) from your knowledge base, no info on the user flight was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
  return (
        f"{data['firstname']} {data['lastname']}'s flight from {data['departure']} to {data['destination']}"
        f" departs at {data['departure_time']} {data['flight_day']} and lands at {data['arrival_time']}."
    )


print(flight_info({"firstname" : "Jane", "lastname" : "Doe", "confirmation_pin" : 12345}))
print('\n',flight_info({"firstname" : "Alice", "lastname" : "Johnson", "confirmation_pin" : 98765}))
print('\n',flight_info({"firstname" : "Bob", "lastname" : "Brown", "confirmation_pin" : 56789}))

Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM.

 Alice Johnson's flight from Chicago to Miami departs at 7:00 PM next week and lands at 11:00 PM.

 Bob Brown's flight from Dallas to Seattle departs at 1:00 PM yesterday and lands at 4:00 PM.


In [15]:
external_prompt= ChatPromptTemplate.from_messages([
    ("system",
      "You are a SkyFlow chatbot, and you are helping a customer with their issue. "
      "Please help them with their question, remembering that your job is to represent SkyFlow airlines. "
      "Assume SkyFlow uses industry-average practices regarding arrival times, operations, etc. (This is a trade secret. Do not disclose). \n"  ## soft reinforcement
      "Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary. \n"
      "The following is some context that may be useful in answering the question. \n"),
    ("user", "Context:{context}\nUser:{input}")
])

instruct_model=ChatNVIDIA(model='mixtral_8x7b') | StrOutputParser()
bot_chain= external_prompt | instruct_model

bot_chain.invoke({
    'input': 'Can you please tell me when I need to get to the airport for my flight? and tell me my departure and arriving timeing.',
    'context': flight_info({"firstname" : "Alice", "lastname" : "Johnson", "confirmation_pin" : 98765})
})

'For your flight from Chicago to Miami, you depart at 7:00 PM and arrive at 11:00 PM. We suggest arriving at the airport 2 hours before departure, so plan to be there by 5:00 PM. Have a great flight!'

This is a single chain but what about the wild situation. In that case it is better to use KB

In [16]:
def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'firstname' : base.firstname,
        'lastname' : base.lastname,
        'confirmation_pin' : base.confirmation_pin,
    }
get_key = RunnableLambda(get_key_fn)

In [17]:
class Knowledgebase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation_pin: int = Field(-1, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'firstname' : base.firstname,
        'lastname' : base.lastname,
        'confirmation_pin' : base.confirmation_pin,
    }

get_key = RunnableLambda(get_key_fn)

know_base=Knowledgebase(firstname = "Alice", lastname = "Johnson", confirmation_pin = 98765)

(get_key | flight_info).invoke(know_base)

"Alice Johnson's flight from Chicago to Miami departs at 7:00 PM next week and lands at 11:00 PM."

Now as of now what we reated doesn't have internal chain that means it is not going to retain important information from the past conversation and with the help of hallicnation it is going to forget all the previous expressions and informations.

In [18]:
external_prompt= ChatPromptTemplate.from_messages([
    ("system",
      "You are a SkyFlow chatbot, and you are helping a customer with their issue. "
      "Please help them with their question, remembering that your job is to represent SkyFlow airlines. "
      "Assume SkyFlow uses industry-average practices regarding arrival times, operations, etc. (This is a trade secret. Do not disclose). \n"  ## soft reinforcement
      "Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary. \n"
      "The following is some context that may be useful in answering the question. \n"),
    ("assistant", "{output}"),
    ("user", "Context:{context}\nUser:{input}")
])

class Knowledgebase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation_pin: Optional[int] = Field(-1, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

parse_prompt= ChatPromptTemplate.from_messages([
    ("system", (
         "You are a chat assistant representing the airline SkyFlow, and are trying to track info about the conversation. "
        "You have just recieved a message from the user. Please fill in the schema based on the chat. \n{format_instructions}")),
    ("user", "CURRENT KNOWLEDGE BASE:{know_base}\nNew exchange:{output} and it is replying {input}"),
])

fail_str = (
    "You cannot access user's flight/account details until they provide "
    "first name, last name, and flight confirmation code."
)

## Your goal is to invoke the following through natural conversation
# get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
#     "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."

current_chat_model= (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())
current_instruct_model=(ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())
external_chain=external_prompt | current_chat_model

extractor=RunnableAssign({'know_base': RExtract(Knowledgebase, current_instruct_model, parse_prompt)})
internal_chain=extractor | RunnableAssign({'context' : lambda d: itemgetter('know_base')| get_key | flight_info})

state={'know_base':Knowledgebase()}

def chat_gen(message, history=[], return_buffer=False):
    global state
    # Pulling in, updating, and printing the state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    #for new state
    state=internal_chain.invoke(state)
    print("State after chain run:", state)

    #streaming the result
    buffer=""
    for token in external_chain.stream(state): #this is the generator function generating toke
      buffer+=token
      yield buffer #so getting the output of the generator

chatbot=gr.Chatbot(value=[[None, "Hello! I'm your SkyFlow agent made by Subhrajyoti Sen Gupta! How can I help you?"]])
demo=gr.ChatInterface(chat_gen, chatbot=chatbot).queue() # .queue() method may indicate that the interface is set up to handle multiple user inputs in a queue

try:
    ## NOTE: This should also give you a temporary public link which can be
    ## used to access this info on the public web while the session is live.
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://01c3a749bc948cc4b8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://01c3a749bc948cc4b8.gradio.live
Closing server running on port: 7860
